In [1]:
import torch
import numpy as np
from qmc.tracehess import autograd_trace_hessian
from torch import nn, optim

In [2]:
class NelectronVander(nn.Module):
    #ansatz given by the Vandermonde determinant of the one electron wavefunctions e^(-alpha_i*r_i)
    #input is 1D tensor which determines the number of particles (i.e. the dimension)
    def __init__(self, alpha):
        super(NelectronVander, self).__init__()
        self.alpha = nn.Parameter(alpha)
    
    def forward(self, x):
        #returns the log prob. of the wavefunction
        #input is tensor of size m x alpha.size or m x n x alpha.size
        a = torch.exp(-self.alpha*x.unsqueeze(-1)) - torch.exp(-self.alpha*x.unsqueeze(-2))
        return 2 * torch.sum(torch.log(torch.abs(a[...,torch.triu(torch.ones(a.shape[-1],a.shape[-1]), diagonal=1).nonzero(as_tuple = True)[0],torch.triu(torch.ones(5,5), diagonal=1).nonzero(as_tuple = True)[1] ])),-1)
    
    
    def wave(self,x):
        # Returns the value of the wavefunction
        #input is tensor of size m x alpha.size or m x n x alpha.size
        a = torch.exp(-self.alpha*x.unsqueeze(-1)) - torch.exp(-self.alpha*x.unsqueeze(-2))
        return torch.prod(a[...,torch.triu(torch.ones(a.shape[-1],a.shape[-1]), diagonal=1).nonzero(as_tuple = True)[0],torch.triu(torch.ones(5,5), diagonal=1).nonzero(as_tuple = True)[1] ],-1)
    
    
        

In [3]:
class NelectronVanderWithMult(nn.Module):
    #ansatz given by the Vandermonde determinant of the one electron wavefunctions e^(-alpha_i * r_i) 
    #multiplied by e^(-beta * (r_1 + r_2 + ... r_N))
    #input is alpha, beta where alpha is 1D tensor which determines the number of particles and beta is scalar
    def __init__(self, alpha, beta):
        super(NelectronVanderWithMult, self).__init__()
        self.alpha = nn.Parameter(alpha)
        self.beta = nn.Parameter(beta)
    
    def forward(self, x):
        #returns the log prob. of the wavefunction
        #input is tensor of size m x alpha.size or m x n x alpha.size
        a = torch.exp(-self.alpha*x.unsqueeze(-1)) - torch.exp(-self.alpha*x.unsqueeze(-2))
        return 2 * ( -self.beta * torch.sum(x, -1)
            + torch.sum(torch.log(torch.abs(a[...,torch.triu(torch.ones(a.shape[-1],a.shape[-1]), diagonal=1).nonzero(as_tuple = True)[0],torch.triu(torch.ones(5,5), diagonal=1).nonzero(as_tuple = True)[1] ])),-1) )
    
    
    def wave(self,x):
        # Returns the value of the wavefunction
        #input is tensor of size m x alpha.size or m x n x alpha.size
        a = torch.exp(-self.alpha*x.unsqueeze(-1)) - torch.exp(-self.alpha*x.unsqueeze(-2))
        return torch.exp(-self.beta * torch.sum(x, -1)) * torch.prod(a[...,torch.triu(torch.ones(a.shape[-1],a.shape[-1]), diagonal=1).nonzero(as_tuple = True)[0],torch.triu(torch.ones(5,5), diagonal=1).nonzero(as_tuple = True)[1] ],-1)
    
    

In [5]:
f = NelectronVander(torch.rand(5))

In [6]:
f(torch.rand((3,5)))

tensor([-58.3537, -47.6753, -50.1161], grad_fn=<MulBackward0>)

In [7]:
f.wave(torch.rand((3,7,5)))

tensor([[ 7.7231e-10, -2.9062e-16,  8.7250e-13,  1.8854e-12, -2.2168e-17,
          5.7117e-14, -5.4074e-10],
        [ 5.8062e-10, -2.8236e-10, -5.8728e-14, -3.1776e-12, -5.4689e-11,
         -3.7469e-12,  3.7089e-13],
        [ 2.6654e-11,  5.0569e-12, -5.6491e-12,  1.0574e-14,  3.6131e-12,
         -4.9153e-12,  1.1617e-11]], grad_fn=<ProdBackward1>)

In [8]:
x = torch.rand(4,6,5)

In [9]:
f = NelectronVanderWithMult(torch.rand(5),torch.rand(1))

In [10]:
f.wave(torch.rand((3,7,5)))

tensor([[ 1.6081e-11,  9.3768e-13, -1.3205e-10, -4.5301e-10,  1.3010e-09,
         -2.6110e-11,  1.3874e-11],
        [ 9.7023e-15, -4.8613e-11, -6.5874e-11, -3.5132e-11, -1.1234e-12,
         -7.5927e-10,  2.9838e-11],
        [-1.4216e-10,  6.1855e-11, -2.8439e-11,  7.5238e-11, -1.8356e-10,
          3.3950e-11,  2.0195e-15]], grad_fn=<MulBackward0>)

In [11]:
f(torch.rand((9,5)))

tensor([-51.4866, -50.4453, -45.3356, -45.0058, -59.7260, -47.8277, -45.8100,
        -59.9616, -52.4779], grad_fn=<MulBackward0>)